In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../modules")

from pathlib import Path
from transformers import TrainingArguments
from trl import SFTTrainer

from common import constants
from common.data import finqa
from training import models

/home/pauliusztin/anaconda3/envs/hands-on-llms/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Datasets

In [3]:
ROOT_DATASET_DIR = Path("../dataset")

In [4]:
training_dataset = finqa.FinQADataset(
    data_path=ROOT_DATASET_DIR / "train.json", scope = constants.Scope.TRAINING
    ).to_huggingface()
training_dataset

Dataset({
    features: ['text'],
    num_rows: 6251
})

In [5]:
training_dataset[0]

{'text': '        ###System: You are a professional financial advisor. Your task is to read a financial report and answer the given question.\n         The financial report you will get will be in the following format:\n         ###START_FINANCIAL_REPORT\n         ###PRE_TEXT:\n         <some text>\n         ###TABLE:\n         <table with the financial data>\n         ###POST_TEXT:\n         <some text>\n         ###END_FINANCIAL_REPORT\n         Your question from the human will be in the following format:\n         ###Human:\n        ###QUESTION: <question>\n         Your answer as an assisant should be in the following format:\n         ###Assistant:\n        ###ANSWER: <answer>\n         You should output all the reasoning steps in the following format:\n         ###STEP 0: <arg1> <operation> <arg2> = <result #0> \n         ###STEP 1: <arg1> <operation> <arg2> = <result #1>\n         ...        ###STEP N:<arg1> <operation> <arg2> = <result #N> \n         You should also output the

In [6]:
validation_dataset = finqa.FinQADataset(
    data_path=ROOT_DATASET_DIR / "test.json", scope=constants.Scope.VALIDATION
).to_huggingface()
validation_dataset

Dataset({
    features: ['text'],
    num_rows: 1147
})

In [7]:
validation_dataset[0]

{'text': '        ###System: You are a professional financial advisor. Your task is to read a financial report and answer the given question.\n         The financial report you will get will be in the following format:\n         ###START_FINANCIAL_REPORT\n         ###PRE_TEXT:\n         <some text>\n         ###TABLE:\n         <table with the financial data>\n         ###POST_TEXT:\n         <some text>\n         ###END_FINANCIAL_REPORT\n         Your question from the human will be in the following format:\n         ###Human:\n        ###QUESTION: <question>\n         Your answer as an assisant should be in the following format:\n         ###Assistant:\n        ###ANSWER: <answer>\n         You should output all the reasoning steps in the following format:\n         ###STEP 0: <arg1> <operation> <arg2> = <result #0> \n         ###STEP 1: <arg1> <operation> <arg2> = <result #1>\n         ...        ###STEP N:<arg1> <operation> <arg2> = <result #N> \n         You should also output the

## Train Model

In [8]:
RESULT_DIR = Path("../results/finqa")

In [9]:
model, tokenizer, peft_config = models.build_qlora_model()

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.22s/it]


In [10]:
training_arguments = TrainingArguments(
    output_dir=str(RESULT_DIR),
    per_device_train_batch_size=4,  # increase this value if you have more VRAM
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",  # This parameter activate QLoRa's pagination
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=1,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

In [ ]:
model.config.use_cache = (
    False  # Gradient checkpointing is used by default but not compatible with caching
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)
trainer.train()

## Test Model

In [ ]:
loaded_model, loaded_tokenizer, _ = models.build_qlora_model(
    peft_model_id="results/checkpoint-10"
)

In [ ]:
print(
    models.prompt(
        model=loaded_model,
        tokenizer=loaded_tokenizer,
        input_text=validation_dataset["text"][0],
    )
)